In [ ]:
class LunaTrainingApp:
   def __init__(self, sys_argv=None):
        if sys_argv is None:
            sys_argv = sys.argv[1:]

        parser = argparse.ArgumentParser()
        parser.add_argument('--num-workers',help='Number of worker processes for background data loading',
                           default = 8,
                           type = int,)

     ....
     self.cli_args = parser.parse_args(sys_argv)
     self.time_str = datetime.datetime.now().strftime('%Y-%m-%d_%H,%M,%S')
     ....

#这里是一个日志的记录
def main(self):
    # log.info("Strating {}, {}".format(type(self)._name_,self.cli_args))
    log.info(f"Initialized {type(self).__name__} with args: {self.cli_args}")

    ....
if __name__ == '__main__':
    LunaTrainingApp.main()
    

In [ ]:
import datetime
from util.util import importstr
from util.logconf import logging
log = logging.getLogger('nb')

def run(app,*argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=4')
    # log.info("Running: {} ({!r}).main()".format(app,argv))
    log.info(f"Running: {app} ({argv!r}).main()")

    app_cls = importstr(*app.rsplit('.',1))
    app_cls(argv).main()

    # log.info("Finished: {}({!r}).main()".format(app,argv))
    log.info(f"Finished: {app}({argv!r}).main()")
    

In [ ]:
run('LunaTrainingApp','--num-workers=4')

1:gpu
2:模型初始化
3：优化器定义

In [ ]:
class LunaTrainingApp:
    def __init__(self,sys_argv=None):
        # ....
        # self.use_cuda = torch.cuda.is_available()
        # self.device = torch.device("cuda" if self.use_cuda else "cpu")

        ...
        if torch.backends.mps.is_available():  # 检查 Metal API 是否可用
            self.device = torch.device('mps')  # 使用 Metal API (仅限 Apple Silicon)
            log.info(f"Using Metal API on Apple Silicon devices.")
        elif torch.cuda.is_available():  # 检查 CUDA 是否可用（适用于支持 CUDA 的设备）
            self.device = torch.device("cuda")
            log.info(f"Using CUDA: {torch.cuda.device_count()} devices.")
        else:
            self.device = torch.device("cpu")  # 如果没有 CUDA 或 Metal API，使用 CPU
            log.info(f"Using CPU.")

        self.model = self.initModel()
        selff.optimizer = self.initOptimizer()

     # def initModel(self):
     #     model = LunaModel()
     #     if self.use_cuda:
     #         # log.info("Using CUDA: {} devices.".format(torch.cuda.device_count()))
     #           log.info(f"Using CUDA: {torch.cuda.device_count()} devices.")
     #         if torch.cuda.device_count() > 1:
     #             model = nn.DataParallel(model)
     #         model = model.to(self.device)
     #    return model

       def initModel(self):
        model = LunaModel()
        if self.device.type == 'cuda' or self.device.type == 'mps':  # 检查设备类型
            log.info(f"Using device: {self.device}.")
            if torch.cuda.device_count() > 1 and self.device.type == 'cuda':  # 支持多GPU时使用 DataParallel
                model = nn.DataParallel(model)
            model = model.to(self.device)
        return model

     def initOptimizer(self):
         return SGD(self.model.parameters(),lr = 0.001,momentum=0.99)
         # return Adam(self.mode.parameters())

In [ ]:
数据加载器

In [ ]:
# def initTrainDl(self):
#     train_ds = LunaDataset(
#     val_stride=10,
#     isValSet_bool=False,
#     )
#     batch_size = self.cli_args.batch_size

#     if self.use_cuda:
#         batch_size *= torch.cuda.device_count()
        
#     train_dl = DataLoader(
#         train_ds,
#         batch_size=batch_size,
#         num_workers=self.cli_args.num_workers,
#         pin_memory = self.use_cuda,
#     )    

#     return train_dl

# def initValDl(self):
#     val_ds = LunaDataset(
#         val_stride=10,
#         isValSet_bool=True,
#     )
#     batch_size = self.cli_args.batch_size

#     if self.use_cuda:
#         batch_size *= torch.cuda.device_count()
        
#     val_dl = DataLoader(
#         val_ds,
#         batch_size=batch_size,
#         num_workers=self.cli_args.num_workers,
#         pin_memory = self.use_cuda,
#     )    

#     return val_dl

In [ ]:
def initTrainDl(self):
    train_ds = LunaDataset(
        val_stride=10,
        isValSet_bool=False,
    )
    batch_size = self.cli_args.batch_size

    # 直接设置 batch_size 和 pin_memory，不做设备判断
    batch_size *= 1  # 对于苹果电脑，这里我们不需要调整 batch_size
    pin_memory = False  # 禁用 pin_memory，因为没有 CUDA 或 Metal 加速

    train_dl = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=self.cli_args.num_workers,
        pin_memory=pin_memory,
    )    

    return train_dl

def initValDl(self):
    val_ds = LunaDataset(
        val_stride=10,
        isValSet_bool=True,
    )
    batch_size = self.cli_args.batch_size

    # 直接设置 batch_size 和 pin_memory，不做设备判断
    batch_size *= 1  # 对于苹果电脑，这里我们不需要调整 batch_size
    pin_memory = False  # 禁用 pin_memory，因为没有 CUDA 或 Metal 加速

    val_dl = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=self.cli_args.num_workers,
        pin_memory=pin_memory,
    )    

    return val_dl


In [ ]:
def main(self):
    # log.info("Strating {}, {}".format(type(self)._name_,self.cli_args))
    log.info(f"Initialized {type(self).__name__} with args: {self.cli_args}")
    train_dl = self.initTrainDl()
    val_dl = self.initValDl()
    


模型核心部分

In [ ]:
class LunaBlock(nn.Module):
    def __init__(self,in_channels,conv_channels):
        super().__init__()

        self.conv1 = nn.Conv3d(in_channels,conv_channels,kernel_size=3,padding=1,bias=True)
        self.relu1 = nn.ReLU(inplace = True)

        # self.conv2 = nn.Conv3d(in_channels,conv_channels,kernel_size=3,padding=1,bias=True)
         self.conv2 = nn.Conv3d(conv_channels, conv_channels, kernel_size=3, padding=1, bias=True)
        self.relu2 = nn.ReLU(inplace = True)

        self.maxpool = nn.MaxPool3d(2,2)

     def forward(self,input_batch):
         block_out = self.conv1(input_batch)
         block_out = self.relu1(block_out)
         block_out = self.conv2(block_out)
         block_out = self.relu2(block_out)
         block_out = self.maxpool(block_out)

         return block_out


In [ ]:
import math

class LunaModel(nn.Module):
    def __init__(self,in_channels=1, conv_channels=8):
        super().__init__()

        self.tail_batchnorm = nn.BatchNorm3d(1)
        
        self.block1 = LunaBlock(in_channels, conv_channels)
        self.block2 = LunaBlock(conv_channels, conv_channels*2)
        self.block3 = LunaBlock(in_channels * 2, conv_channels*4)
        self.block4 = LunaBlock(in_channels * 4, conv_channels*8)

        self.head_linear = nn.Linear(1152,2)
        # self.head_linear = nn.Linear(conv_channels * 8 * 4 * 4 * 4, 2)

        self.head_softmax = nn.Softmax(dim = 1)
        
        self.__init__weights()

    def forward(self, input_batch):
        bn_output = self.tail_batchnorm(input_batch)
        
        block_out = self.block1(bn_output)
        block_out = self.block2(block_out)
        block_out = self.block3(block_out)
        block_out = self.block4(block_out)
        
        conv_flat = block_out.view(block_out.size(0),-1)
        linear_output = self.head_linear(conv_flat)
        return linear_output,self.head_softmax(linear_output)

     def __init__weights(self):
         for m in self.modules():
             if type(m) in (nn.Linear,nn.Conv3d,nn.Conv2d,nn.ConvTranspose2d,nn.ConvTranspose3d):
                 nn.init.kaiming_normal_(
                     m.weight.data, a=0, mode='fan_out',nonlinearity = 'relu',
                 )

                 if m.bias is not None:
                     fan_in,fan_out = nn.init._calculate_fan_in_and_out(m.weight.data)
                     bound = 1 / math.sqrt(fan_out)
                     nn.init.normal_(m.bias,-bound,bound)

定义损失计算 和 训练验证环节

In [ ]:
# def main(self):
#     # log.info("Strating {}, {}".format(type(self)._name_,self.cli_args))
#     log.info(f"Initialized {type(self).__name__} with args: {self.cli_args}")
#     train_dl = self.initTrainDl()
#     val_dl = self.initValDl()

#     for epoch_ndx in range(1,self.cli_args.epochs + 1):
#         log.info("Epoch {} of {}, {}/{} batches of size {} * {}".format(
#             epoch_ndx,
#             self.cli_args,epochs,
#             len(train_dl),
#             len(val_dl),
#             self.cli_args.batch_size,
#             (torch.cuda.device_count() if self.use_cuda else 1),
#         ))

#         trnMetrics_t = self.doTraining(epoch_ndx, val_dl)
#         self.logMetrics(epoch_ndx, 'trn',trnMetrics_t)

#         valMetrics_t = self.doValidation(epoch_ndx, val_dl)
#         self.logMetrics(epoch_ndx, 'val', valMetrics_t)
    

In [ ]:
def main(self):
    # log.info("Strating {}, {}".format(type(self)._name_,self.cli_args))
    log.info(f"Initialized {type(self).__name__} with args: {self.cli_args}")
    train_dl = self.initTrainDl()
    val_dl = self.initValDl()

    for epoch_ndx in range(1, self.cli_args.epochs + 1):
        log.info("Epoch {} of {}, {}/{} batches of size {} * {}".format(
            epoch_ndx,
            self.cli_args.epochs,
            len(train_dl),
            len(val_dl),
            self.cli_args.batch_size,
            1,  # Only 1 device (CPU) on Apple machines
        ))

        trnMetrics_t = self.doTraining(epoch_ndx, val_dl)
        self.logMetrics(epoch_ndx, 'trn', trnMetrics_t)

        valMetrics_t = self.doValidation(epoch_ndx, val_dl)
        self.logMetrics(epoch_ndx, 'val', valMetrics_t)


In [ ]:
METRICS_LABEL_NDX = 0
METRICS_PRED_NDX = 1
METRICS_LOSS_NDX = 2
METRICS_SIZE = 3
def doTraining(self,epoch_ndx,train_dl):
    self.model.train()

    trnMetrics_g = torch.zeros(
        METRICS_SIZE,
        len(train_dl.dataset),
        device = self.device
    )

    batch_iter = enumerateWithEstimate(
        train_dl,
        "E{} Training".format(epoch_ndx),
        start_ndx = train_dl.num_workers,
    )

    for batch_ndx, batch_tup in batch_iter:
        self.optimizer.zero_grad()

        loss_var = self.computeBatchLoss(
            batch_ndx,
            batch_tup,
            train_dl.batch_size,
            trnMetrics_g
        )

        loss_var.backward()
        self.optimizer.step()
    self.totalTrainingSamples_count += len(train_dl.dataset)
    return trnMetrics_g.to('cpu')
        
    

In [ ]:
def doValidation(self,epoch_ndx,train_dl):
    with torch.no_grad():
        self.model.eval()
        
        valMetrics_g = torch.zeros(
            METRICS_SIZE,
            len(val_dl.dataset),
            device = self.device
        )

        # batch_iter = enumerateWithEstimate(
        #     val_dl,
        #     f"E{epoch_ndx} Validation",
        #     start_ndx=val_dl.num_workers,
        # )

         for batch_ndx, batch_tup in batch_iter:
             self.computeBatchLoss(
                  batch_ndx,
                  batch_tup,
                  val_dl.batch_size,
                  valMetrics_g
             )

     return valMetrics_g.to('cpu')
        
    

In [ ]:
def computeBatchLoss(self,batch_ndx,batch_tup,batch_size,metrics_g):
    input_t, label_t, _series_list, _center_list = batch_tup

    input_g = input_t.to(self.device,non_block=True)
    label_g = label_t.to(self.device,non_block=True)

    logits_g,probability_g = self.model(input_g)

    loss_func = nn.CrossEntropyLoss(reduction='none')

    loss_g = loss_func(logits_g, label_g[:,1])

    start_ndx = batch_ndx * batch_size
    end_ndx = start_ndx + label_t.size(0)

    metrics_g[METRICS_LABEL_NDX,start_ndx:end_ndx] = label_g[:,1].detach()
    metrics_g[METRICS_PRED_NDX,start_ndx:end_ndx] = probability_g[:,1].detach()
    metrics_g[METRICS_LOSS_NDX,start_ndx:end_ndx] = loss_g.detach()

    return loss_g.mean()

在日志中保存重要信息

In [ ]:
def logMetrics(self,epoch_ndx,mode_str,metrics_t,classificationThreshold = 0.5):
    self.initTensorboardWriters()
    log.info("E{} {}".format(epoch_ndx, type(self).__name__))

    negLabel_mask = metrics_t[METRICS_LABEL_NDX] <= classificationThreshold
    negPred_mask = metrics_t[METRICS_PRED_NDX] <= classificationThreshold

    posLabel_mask = ~negLabel_mask
    posPred_mask = ~negPred_mask

    neg_count = int((negLabel_mask & negPred_mask).sum())
    pos_count = int((posLabel_mask & posPred_mask).sum())

    metrics_dict = {}
    metrics_dict['loss/all'] = metrics_t[METRICS_LOSS_NDX].mean()
    metrics_dict['loss/neg'] = metrics_t[METRICS_LOSS_NDX,negLabel_mask].mean()
    metrics_dict['loss/pos'] = metrics_t[METRICS_LOSS_NDX,posLabel_mask].mean()

    metrics_dict['correct/all'] = (pos_correct + neg_correct) / np.float32(metrics_t.shape[1]) * 100
    metrics_dict['correct/neg'] = neg_correct / np.float32(neg_count) * 100
    metrics_dict['correct/pos'] = pos_correct / np.float32(pos_count) * 100

    log.info(("E{} {:8} {loss/all:,4f} loss," + "{correct/all:-5.1f}% correct,").format(epoch_ndx, mode_str, **metrics_dict))
    log.info(("E{} {:8} {loss/neg:,4f} loss," + "{correct/neg:-5.1f}% correct ({neg_correct:} of {neg_count:})").format(epoch_ndx, mode_str+'_neg',neg_correct=neg_correct,neg_count=neg_count, **metrics_dict))
    log.info(("E{} {:8} {loss/pos:,4f} loss," + "{correct/pos:-5.1f}% correct ({neg_correct:} of {neg_count:})").format(epoch_ndx, mode_str+'_pos',pos_correct=pos_correct,pos_count=pos_count, **metrics_dict))

    writer = getattr(self,mode_str + '_writer')
    for key,value in metrics_dict.items():
        writer.add_scalar(key, value,self.totalTrainingSamples_count)
    writer.add_pr_curve('pr',metrics_t[METRICS_LABEL_NDX], metrics_t[METRICS_PRED_NDX],self.totalTrainingSamples_count)

        bin = [x/50.0 for x in range(51)]

        negHist_mask = negLabel_mask & (metrics_t[METRICS_PRED_NDX] > 0.01)
        posHist_mask = posLabel_mask & (metrics_t[METRICS_PRED_NDX] < 0.99)

        if  negHist_mask.any():
            writer.add_histogram('is_neg', metrics_t[METRICS_PRED_NDX,negHist_mask],self.totalTrainingSamples_count,bins=bins)

        if  posHist_mask.any():
            writer.add_histogram('is_pos', metrics_t[METRICS_PRED_NDX,posHist_mask],self.totalTrainingSamples_count,bins=bins)
            